In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from nlpsig_networks.scripts.lstm_baseline_functions import (
    lstm_hyperparameter_search,
    obtain_path,
)

In [4]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [5]:
%run ../load_anno_mi.py

In [6]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime,speaker
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-10-02 00:00:13,-1
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-10-02 00:00:24,1
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-10-02 00:00:25,-1
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-10-02 00:00:34,1
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-10-02 00:00:34,-1


In [7]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)

sbert_embeddings.shape

(9699, 384)

# Baseline: LSTM classification

In [10]:
num_epochs = 100
hidden_dim_sizes = [200, 300, 384]
num_layers = 1
bidirectional = True
dropout_rates = [0.1, 0.2]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

In [11]:
kwargs = {
    "num_epochs": num_epochs,
    "df": anno_mi,
    "id_column": "transcript_id",
    "label_column": "client_talk_type",
    "embeddings": sbert_embeddings,
    "y_data": y_data_client,
    "output_dim": output_dim_client,
    "hidden_dim_sizes": hidden_dim_sizes,
    "num_layers": num_layers,
    "bidirectional": bidirectional,
    "dropout_rates": dropout_rates,
    "learning_rates": learning_rates,
    "seeds": seeds,
    "loss": loss,
    "gamma": gamma,
    "device": device,
    "path_indices": client_index,
    "split_ids": client_transcript_id,
    "k_fold": True,
    "patience": patience,
    "validation_metric": validation_metric,
    "verbose": False,
}

## history_length=5

In [12]:
size = 5

In [13]:
(
    bilstm_history_5_kfold,
    best_bilstm_history_5_kfold,
    _,
    __,
) = lstm_hyperparameter_search(
    history_lengths=[size],
    results_output=f"{output_dir}/lstm_history_{size}_focal_{gamma}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/lstm_history_5_focal_2_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/lstm_history_5_focal_2_kfold_best_model.csv


In [14]:
bilstm_history_5_kfold.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

/tmp/ipykernel_3407032/3415036038.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  bilstm_history_5_kfold.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()


accuracy        f1  precision  \
hidden_dim dropout_rate learning_rate                                  
200        0.1          0.0001         0.634547  0.519613   0.512197   
                        0.0005         0.628544  0.519520   0.510432   
                        0.0010         0.622222  0.515454   0.506094   
           0.2          0.0001         0.636079  0.519779   0.512261   
                        0.0005         0.629694  0.520027   0.510934   
                        0.0010         0.627714  0.517222   0.509453   
300        0.1          0.0001         0.632759  0.519478   0.511447   
                        0.0005         0.621711  0.515842   0.507247   
                        0.0010         0.629502  0.516911   0.509068   
           0.2          0.0001         0.633014  0.518748   0.511068   
                        0.0005         0.624840  0.518100   0.509662   
                        0.0010         0.629183  0.517299   0.508492   
384        0.1          0.0001         0.634547  0.515277   0.509226   
                        0.0005         0.627203  0.516900   0.510293   
                        0.0010         0.647254  0.522685   0.518117   
           0.2          0.0001         0.634483  0.515202   0.509171   
                        0.0005         0.629055  0.518017   0.510911   
                        0.0010         0.629821  0.516384   0.508506   

                                         recall  valid_accuracy  valid_f1  \
hidden_dim dropout_rate learning_rate                                       
200        0.1          0.0001         0.536266        0.665382  0.563706   
                        0.0005         0.540885        0.667737  0.570114   
                        0.0010         0.539096        0.662331  0.569553   
           0.2          0.0001         0.534982        0.666131  0.563174   
                        0.0005         0.540139        0.668058  0.569780   
                        0.0010         0.539868        0.664793  0.570238   
300        0.1          0.0001         0.538203        0.667255  0.566536   
                        0.0005         0.539732        0.662653  0.567789   
                        0.0010         0.535622        0.668219  0.572684   
           0.2          0.0001         0.537209        0.666881  0.565415   
                        0.0005         0.541077        0.664900  0.568798   
                        0.0010         0.534887        0.670039  0.573476   
384        0.1          0.0001         0.531036        0.668861  0.564392   
                        0.0005         0.539552        0.663937  0.567264   
                        0.0010         0.532861        0.677639  0.571178   
           0.2          0.0001         0.530733        0.669236  0.564509   
                        0.0005         0.539540        0.663562  0.564704   
                        0.0010         0.534401        0.665061  0.568381   

                                       valid_precision  valid_recall    k  \
hidden_dim dropout_rate learning_rate                                       
200        0.1          0.0001                0.558355      0.574355  5.0   
                        0.0005                0.562542      0.583625  5.0   
                        0.0010                0.560306      0.586769  5.0   
           0.2          0.0001                0.558015      0.572297  5.0   
                        0.0005                0.562439      0.582488  5.0   
                        0.0010                0.562381      0.586919  5.0   
300        0.1          0.0001                0.560477      0.577860  5.0   
                        0.0005                0.559548      0.584586  5.0   
                        0.0010                0.565082      0.586446  5.0   
           0.2          0.0001                0.559704      0.576238  5.0   
                        0.0005                0.561221      0.584541  5.0   
                        0.0010                0.565575  

In [15]:
best_bilstm_history_5_kfold

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,bidirectional,hidden_dim,dropout_rate,learning_rate,seed,loss_function,gamma,k_fold,n_splits,batch_size
0,None,0.630843,0.516468,"[0.7559266438049799, 0.42970379641218187, 0.36...",0.508000,"[0.7802400738688827, 0.4345991561181435, 0.309...",0.533273,"[0.7330827067669173, 0.4249174917491749, 0.441...",None,0.678227,...,True,300,0.2,0.001,1,focal,2,True,5,64
0,None,0.625862,0.517632,"[0.7482302246845183, 0.4744838332684067, 0.330...",0.508090,"[0.7996710526315789, 0.44944649446494467, 0.27...",0.539403,"[0.7030075187969925, 0.5024752475247525, 0.412...",None,0.664258,...,True,300,0.2,0.001,12,focal,2,True,5,64
0,None,0.630843,0.517798,"[0.750037397157816, 0.4557988645579887, 0.3475...",0.509387,"[0.7768825534552216, 0.4481658692185008, 0.303...",0.531985,"[0.7249855407750144, 0.4636963696369637, 0.407...",None,0.667630,...,True,300,0.2,0.001,123,focal,2,True,5,64


In [16]:
best_bilstm_history_5_kfold["f1"].mean()

0.5172990309232012

In [17]:
best_bilstm_history_5_kfold["precision"].mean()

0.5084921482433392

In [18]:
best_bilstm_history_5_kfold["recall"].mean()

0.5348870063408886

In [19]:
np.stack(best_bilstm_history_5_kfold["f1_scores"]).mean(axis=0)

array([0.75139809, 0.45332883, 0.34717017])

In [20]:
np.stack(best_bilstm_history_5_kfold["precision_scores"]).mean(axis=0)

array([0.78559789, 0.44407051, 0.29580804])

In [21]:
np.stack(best_bilstm_history_5_kfold["recall_scores"]).mean(axis=0)

array([0.72035859, 0.46369637, 0.42060606])

## history_length=11

In [22]:
size = 11

In [23]:
(
    bilstm_history_11_kfold,
    best_bilstm_history_11_kfold,
    _,
    __,
) = lstm_hyperparameter_search(
    history_lengths=[size],
    results_output=f"{output_dir}/lstm_history_{size}_focal_{gamma}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/lstm_history_11_focal_2_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/lstm_history_11_focal_2_kfold_best_model.csv


In [24]:
bilstm_history_11_kfold.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

/tmp/ipykernel_3407032/2392464154.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  bilstm_history_11_kfold.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()


accuracy        f1  precision  \
hidden_dim dropout_rate learning_rate                                  
200        0.1          0.0001         0.627842  0.509209   0.502304   
                        0.0005         0.641890  0.527023   0.519033   
                        0.0010         0.630779  0.521670   0.513013   
           0.2          0.0001         0.627714  0.509671   0.503125   
                        0.0005         0.637931  0.524105   0.515574   
                        0.0010         0.627011  0.521863   0.512302   
300        0.1          0.0001         0.618455  0.505811   0.497620   
                        0.0005         0.626373  0.517695   0.508733   
                        0.0010         0.636782  0.524299   0.515492   
           0.2          0.0001         0.622222  0.505860   0.498380   
                        0.0005         0.631545  0.520904   0.511746   
                        0.0010         0.640868  0.525830   0.517928   
384        0.1          0.0001         0.621392  0.509592   0.500969   
                        0.0005         0.633333  0.521875   0.513666   
                        0.0010         0.639144  0.519695   0.513112   
           0.2          0.0001         0.617816  0.506212   0.497928   
                        0.0005         0.640805  0.525211   0.518076   
                        0.0010         0.643103  0.523372   0.516826   

                                         recall  valid_accuracy  valid_f1  \
hidden_dim dropout_rate learning_rate                                       
200        0.1          0.0001         0.522968        0.674160  0.578283   
                        0.0005         0.539136        0.686630  0.589028   
                        0.0010         0.541516        0.678977  0.584832   
           0.2          0.0001         0.522303        0.674695  0.577582   
                        0.0005         0.537357        0.684650  0.591363   
                        0.0010         0.546282        0.672875  0.583264   
300        0.1          0.0001         0.522941        0.669343  0.575946   
                        0.0005         0.536579        0.682777  0.591319   
                        0.0010         0.541142        0.679994  0.583077   
           0.2          0.0001         0.521759        0.670146  0.575205   
                        0.0005         0.537220        0.684811  0.591994   
                        0.0010         0.541096        0.681760  0.584626   
384        0.1          0.0001         0.525201        0.672554  0.579264   
                        0.0005         0.541770        0.682509  0.588698   
                        0.0010         0.531444        0.681225  0.579072   
           0.2          0.0001         0.524696        0.669985  0.577806   
                        0.0005         0.540040        0.687968  0.590499   
                        0.0010         0.534166        0.683098  0.581563   

                                       valid_precision  valid_recall     k  \
hidden_dim dropout_rate learning_rate                                        
200        0.1          0.0001                0.571189      0.589550  11.0   
                        0.0005                0.583319      0.596243  11.0   
                        0.0010                0.576708      0.598811  11.0   
           0.2          0.0001                0.571247      0.587586  11.0   
                        0.0005                0.583466      0.601750  11.0   
                        0.0010                0.573307      0.602862  11.0   
300        0.1          0.0001                0.567465      0.590168  11.0   
                        0.0005                0.582535      0.606281  11.0   
                        0.0010                0.575536      0.594472  11.0   
           0.2          0.0001                0.567670      0.588211  11.0   
                        0.0005                0.583229      0.604981  11.0   
                        0.0010             

In [25]:
best_bilstm_history_11_kfold

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,bidirectional,hidden_dim,dropout_rate,learning_rate,seed,loss_function,gamma,k_fold,n_splits,batch_size
0,None,0.633908,0.521452,"[0.7566034920161169, 0.4368098159509203, 0.370...",0.512451,"[0.7816836262719704, 0.43309002433090027, 0.32...",0.536680,"[0.7330827067669173, 0.4405940594059406, 0.436...",None,0.689306,...,True,300,0.2,0.0005,1,focal,2,True,5,64
0,None,0.637739,0.523214,"[0.7608308605341246, 0.435436690346845, 0.3733...",0.514902,"[0.7812309567336989, 0.441151566469094, 0.3223...",0.538324,"[0.741469057258531, 0.4298679867986799, 0.4436...",None,0.690430,...,True,300,0.2,0.0005,12,focal,2,True,5,64
0,None,0.622989,0.518045,"[0.7429970916883514, 0.4544414792222646, 0.356...",0.507884,"[0.7892682926829269, 0.4223954642097803, 0.311...",0.536655,"[0.7018507807981492, 0.49174917491749176, 0.41...",None,0.674695,...,True,300,0.2,0.0005,123,focal,2,True,5,64


In [26]:
best_bilstm_history_11_kfold["f1"].mean()

0.5209038001451615

In [27]:
best_bilstm_history_11_kfold["precision"].mean()

0.511746071516889

In [28]:
best_bilstm_history_11_kfold["recall"].mean()

0.5372197113677051

In [29]:
np.stack(best_bilstm_history_11_kfold["f1_scores"]).mean(axis=0)

array([0.75347715, 0.44222933, 0.36700492])

In [30]:
np.stack(best_bilstm_history_11_kfold["precision_scores"]).mean(axis=0)

array([0.78406096, 0.43221235, 0.3189649 ])

In [31]:
np.stack(best_bilstm_history_11_kfold["recall_scores"]).mean(axis=0)

array([0.72546751, 0.45407041, 0.43212121])

## history_length=20

In [32]:
size = 20

In [33]:
(
    bilstm_history_20_kfold,
    best_bilstm_history_20_kfold,
    _,
    __,
) = lstm_hyperparameter_search(
    history_lengths=[size],
    results_output=f"{output_dir}/lstm_history_{size}_focal_{gamma}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/lstm_history_20_focal_2_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/lstm_history_20_focal_2_kfold_best_model.csv


In [34]:
bilstm_history_20_kfold.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

/tmp/ipykernel_3407032/3632055421.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  bilstm_history_20_kfold.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()


accuracy        f1  precision  \
hidden_dim dropout_rate learning_rate                                  
200        0.1          0.0001         0.625607  0.500227   0.495526   
                        0.0005         0.631098  0.505785   0.500874   
                        0.0010         0.631865  0.512402   0.506024   
           0.2          0.0001         0.627650  0.500478   0.495652   
                        0.0005         0.635824  0.513603   0.508320   
                        0.0010         0.631354  0.510471   0.504968   
300        0.1          0.0001         0.622669  0.489332   0.486488   
                        0.0005         0.618902  0.500839   0.494087   
                        0.0010         0.635824  0.519582   0.513267   
           0.2          0.0001         0.620179  0.487010   0.484201   
                        0.0005         0.627905  0.509933   0.503023   
                        0.0010         0.634547  0.513422   0.506910   
384        0.1          0.0001         0.630843  0.492849   0.491799   
                        0.0005         0.631801  0.509516   0.504556   
                        0.0010         0.643295  0.520820   0.516710   
           0.2          0.0001         0.631801  0.491318   0.492104   
                        0.0005         0.623946  0.510162   0.502698   
                        0.0010         0.640741  0.522711   0.515627   

                                         recall  valid_accuracy  valid_f1  \
hidden_dim dropout_rate learning_rate                                       
200        0.1          0.0001         0.510305        0.682830  0.591810   
                        0.0005         0.513946        0.703008  0.608725   
                        0.0010         0.525695        0.700707  0.606729   
           0.2          0.0001         0.508388        0.685185  0.591662   
                        0.0005         0.523969        0.699850  0.607165   
                        0.0010         0.521105        0.700546  0.605222   
300        0.1          0.0001         0.494841        0.681974  0.586046   
                        0.0005         0.514720        0.698726  0.608464   
                        0.0010         0.531842        0.696050  0.604557   
           0.2          0.0001         0.492358        0.681171  0.587028   
                        0.0005         0.520357        0.697602  0.607885   
                        0.0010         0.524070        0.703918  0.610539   
384        0.1          0.0001         0.495258        0.687647  0.588180   
                        0.0005         0.521559        0.703757  0.610964   
                        0.0010         0.528449        0.699101  0.601823   
           0.2          0.0001         0.492008        0.687219  0.585090   
                        0.0005         0.528475        0.694230  0.606304   
                        0.0010         0.534646        0.699743  0.606358   

                                       valid_precision  valid_recall     k  \
hidden_dim dropout_rate learning_rate                                        
200        0.1          0.0001                0.583534      0.606088  20.0   
                        0.0005                0.602743      0.617818  20.0   
                        0.0010                0.601194      0.617765  20.0   
           0.2          0.0001                0.584310      0.601949  20.0   
                        0.0005                0.600870      0.618188  20.0   
                        0.0010                0.600391      0.614423  20.0   
300        0.1          0.0001                0.580353      0.595187  20.0   
                        0.0005                0.601007      0.622986  20.0   
                        0.0010                0.598354      0.615786  20.0   
           0.2          0.0001                0.580736      0.596820  20.0   
                        0.0005                0.599711      0.618578  20.0   
                        0.0010             

In [35]:
best_bilstm_history_20_kfold

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,bidirectional,hidden_dim,dropout_rate,learning_rate,seed,loss_function,gamma,k_fold,n_splits,batch_size
0,None,0.645019,0.516117,"[0.7666178623718888, 0.45491143317230276, 0.32...",0.513340,"[0.7763938315539739, 0.4441823899371069, 0.319...",0.519267,"[0.757085020242915, 0.4661716171617162, 0.3345...",None,0.707932,...,True,384,0.1,0.0005,1,focal,2,True,5,64
0,None,0.625096,0.504480,"[0.753204172876304, 0.4363016616957819, 0.3239...",0.499023,"[0.777060270602706, 0.4511013215859031, 0.2689...",0.520161,"[0.7307692307692307, 0.42244224422442245, 0.40...",None,0.701509,...,True,384,0.1,0.0005,12,focal,2,True,5,64
0,None,0.625287,0.507951,"[0.7521214828048235, 0.42680324370465217, 0.34...",0.501306,"[0.7750843817121816, 0.4420866489832007, 0.286...",0.525250,"[0.7304800462695199, 0.41254125412541254, 0.43...",None,0.701830,...,True,384,0.1,0.0005,123,focal,2,True,5,64


In [36]:
best_bilstm_history_20_kfold["f1"].mean()

0.5095157194124188

In [37]:
best_bilstm_history_20_kfold["precision"].mean()

0.5045564266440593

In [38]:
best_bilstm_history_20_kfold["recall"].mean()

0.5215594297042968

In [39]:
np.stack(best_bilstm_history_20_kfold["f1_scores"]).mean(axis=0)

array([0.75731451, 0.43933878, 0.33189387])

In [40]:
np.stack(best_bilstm_history_20_kfold["precision_scores"]).mean(axis=0)

array([0.77617949, 0.44579012, 0.29169967])

In [41]:
np.stack(best_bilstm_history_20_kfold["recall_scores"]).mean(axis=0)

array([0.73944477, 0.43371837, 0.39151515])

## history_length=35

In [42]:
size = 35

In [43]:
(
    bilstm_history_35_kfold,
    best_bilstm_history_35_kfold,
    _,
    __,
) = lstm_hyperparameter_search(
    history_lengths=[size],
    results_output=f"{output_dir}/lstm_history_{size}_focal_{gamma}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/lstm_history_35_focal_2_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/lstm_history_35_focal_2_kfold_best_model.csv


In [44]:
bilstm_history_35_kfold.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

/tmp/ipykernel_3407032/2047737328.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  bilstm_history_35_kfold.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()


accuracy        f1  precision  \
hidden_dim dropout_rate learning_rate                                  
200        0.1          0.0001         0.629566  0.503715   0.499511   
                        0.0005         0.637292  0.513575   0.508597   
                        0.0010         0.619540  0.513413   0.503854   
           0.2          0.0001         0.625798  0.501180   0.496242   
                        0.0005         0.632631  0.514610   0.508108   
                        0.0010         0.636973  0.517218   0.511948   
300        0.1          0.0001         0.618902  0.491224   0.486417   
                        0.0005         0.629502  0.506252   0.501758   
                        0.0010         0.635313  0.516430   0.509901   
           0.2          0.0001         0.611814  0.487290   0.481483   
                        0.0005         0.625607  0.503214   0.497926   
                        0.0010         0.629630  0.515571   0.507766   
384        0.1          0.0001         0.622478  0.490690   0.487602   
                        0.0005         0.626373  0.510425   0.505682   
                        0.0010         0.624904  0.516900   0.508704   
           0.2          0.0001         0.620370  0.493991   0.489109   
                        0.0005         0.614559  0.501053   0.492936   
                        0.0010         0.630651  0.514022   0.510017   

                                         recall  valid_accuracy  valid_f1  \
hidden_dim dropout_rate learning_rate                                       
200        0.1          0.0001         0.510487        0.693320  0.604055   
                        0.0005         0.520569        0.714515  0.624851   
                        0.0010         0.531017        0.700760  0.618480   
           0.2          0.0001         0.508335        0.694337  0.608344   
                        0.0005         0.523968        0.708949  0.621544   
                        0.0010         0.527635        0.709270  0.620284   
300        0.1          0.0001         0.497987        0.688289  0.603881   
                        0.0005         0.514649        0.713712  0.625239   
                        0.0010         0.526153        0.706487  0.614457   
           0.2          0.0001         0.496647        0.684811  0.601920   
                        0.0005         0.514115        0.713177  0.624450   
                        0.0010         0.529638        0.702901  0.617538   
384        0.1          0.0001         0.495186        0.692999  0.604149   
                        0.0005         0.519992        0.709270  0.622619   
                        0.0010         0.532443        0.697870  0.613894   
           0.2          0.0001         0.500910        0.692197  0.604282   
                        0.0005         0.515355        0.705416  0.624108   
                        0.0010         0.522963        0.702259  0.611604   

                                       valid_precision  valid_recall     k  \
hidden_dim dropout_rate learning_rate                                        
200        0.1          0.0001                0.596114      0.616842  35.0   
                        0.0005                0.619403      0.631615  35.0   
                        0.0010                0.606910      0.636674  35.0   
           0.2          0.0001                0.599110      0.623013  35.0   
                        0.0005                0.613405      0.632151  35.0   
                        0.0010                0.614130      0.630522  35.0   
300        0.1          0.0001                0.593414      0.619771  35.0   
                        0.0005                0.619764      0.634425  35.0   
                        0.0010                0.608454      0.622430  35.0   
           0.2          0.0001                0.590758      0.620907  35.0   
                        0.0005                0.618876      0.635300  35.0   
                        0.0010             

In [45]:
best_bilstm_history_35_kfold

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,bidirectional,hidden_dim,dropout_rate,learning_rate,seed,loss_function,gamma,k_fold,n_splits,batch_size
0,None,0.628927,0.508679,"[0.7518529498962349, 0.43203125000000003, 0.34...",0.504573,"[0.7712895377128953, 0.4102373887240356, 0.332...",0.514123,"[0.7333718912666282, 0.45627062706270627, 0.35...",None,0.715318,...,True,300,0.1,0.0005,1,focal,2,True,5,64
0,None,0.632950,0.506217,"[0.7599183197199534, 0.4195680916703393, 0.339...",0.502821,"[0.7666274278987639, 0.4503311258278146, 0.291...",0.517173,"[0.7533256217466744, 0.3927392739273927, 0.405...",None,0.711946,...,True,300,0.1,0.0005,12,focal,2,True,5,64
0,None,0.626628,0.503861,"[0.7521771217712178, 0.424989724619811, 0.3344...",0.497880,"[0.7681640036177269, 0.42342342342342343, 0.30...",0.512652,"[0.7368421052631579, 0.4265676567656766, 0.374...",None,0.713873,...,True,300,0.1,0.0005,123,focal,2,True,5,64


In [46]:
best_bilstm_history_35_kfold["f1"].mean()

0.5062520239641121

In [47]:
best_bilstm_history_35_kfold["precision"].mean()

0.5017578602137986

In [48]:
best_bilstm_history_35_kfold["recall"].mean()

0.514649383195501

In [49]:
np.stack(best_bilstm_history_35_kfold["f1_scores"]).mean(axis=0)

array([0.75464946, 0.42552969, 0.33857692])

In [50]:
np.stack(best_bilstm_history_35_kfold["precision_scores"]).mean(axis=0)

array([0.76869366, 0.42799731, 0.30858261])

In [51]:
np.stack(best_bilstm_history_35_kfold["recall_scores"]).mean(axis=0)

array([0.74117987, 0.42519252, 0.37757576])

## history_length=80

In [52]:
size = 80

In [53]:
(
    bilstm_history_80_kfold,
    best_bilstm_history_80_kfold,
    _,
    __,
) = lstm_hyperparameter_search(
    history_lengths=[size],
    results_output=f"{output_dir}/lstm_history_{size}_focal_{gamma}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/lstm_history_80_focal_2_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/lstm_history_80_focal_2_kfold_best_model.csv


In [54]:
bilstm_history_80_kfold.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

/tmp/ipykernel_3407032/4119866568.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  bilstm_history_80_kfold.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()


accuracy        f1  precision  \
hidden_dim dropout_rate learning_rate                                  
200        0.1          0.0001         0.618199  0.489772   0.485513   
                        0.0005         0.635121  0.511993   0.506233   
                        0.0010         0.633397  0.512416   0.505254   
           0.2          0.0001         0.618646  0.492372   0.487854   
                        0.0005         0.640358  0.516488   0.509973   
                        0.0010         0.638633  0.515572   0.510480   
300        0.1          0.0001         0.617241  0.480904   0.478451   
                        0.0005         0.643167  0.519468   0.513193   
                        0.0010         0.624457  0.511588   0.503399   
           0.2          0.0001         0.616220  0.482043   0.479728   
                        0.0005         0.637612  0.513946   0.507676   
                        0.0010         0.630843  0.512402   0.505059   
384        0.1          0.0001         0.625351  0.490370   0.487170   
                        0.0005         0.634802  0.512558   0.506649   
                        0.0010         0.635632  0.518136   0.511147   
           0.2          0.0001         0.619732  0.486327   0.482560   
                        0.0005         0.637484  0.518413   0.511096   
                        0.0010         0.641890  0.517125   0.511762   

                                         recall  valid_accuracy  valid_f1  \
hidden_dim dropout_rate learning_rate                                       
200        0.1          0.0001         0.495856        0.703704  0.615138   
                        0.0005         0.522438        0.715371  0.628382   
                        0.0010         0.523776        0.717405  0.630219   
           0.2          0.0001         0.498960        0.703329  0.615831   
                        0.0005         0.526178        0.716495  0.629176   
                        0.0010         0.524212        0.718636  0.630154   
300        0.1          0.0001         0.484699        0.702954  0.612209   
                        0.0005         0.528569        0.721312  0.631887   
                        0.0010         0.527395        0.708895  0.626457   
           0.2          0.0001         0.485999        0.704239  0.614374   
                        0.0005         0.524185        0.715639  0.625178   
                        0.0010         0.524285        0.711732  0.624470   
384        0.1          0.0001         0.494524        0.701830  0.612391   
                        0.0005         0.523330        0.713873  0.625344   
                        0.0010         0.531585        0.711678  0.624109   
           0.2          0.0001         0.491455        0.704667  0.614073   
                        0.0005         0.530359        0.708574  0.620274   
                        0.0010         0.526974        0.712802  0.623425   

                                       valid_precision  valid_recall     k  \
hidden_dim dropout_rate learning_rate                                        
200        0.1          0.0001                0.605951      0.630238  80.0   
                        0.0005                0.619434      0.642100  80.0   
                        0.0010                0.621073      0.642260  80.0   
           0.2          0.0001                0.606449      0.630949  80.0   
                        0.0005                0.619664      0.642842  80.0   
                        0.0010                0.623194      0.640097  80.0   
300        0.1          0.0001                0.605451      0.623961  80.0   
                        0.0005                0.624819      0.641549  80.0   
                        0.0010                0.614179      0.645780  80.0   
           0.2          0.0001                0.606147      0.626161  80.0   
                        0.0005                0.617358      0.638099  80.0   
                        0.0010             

In [55]:
best_bilstm_history_80_kfold

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,bidirectional,hidden_dim,dropout_rate,learning_rate,seed,loss_function,gamma,k_fold,n_splits,batch_size
0,None,0.648276,0.523103,"[0.7725055432372506, 0.46400658165364056, 0.33...",0.517029,"[0.790142122769882, 0.46267432321575064, 0.298...",0.532450,"[0.7556390977443609, 0.46534653465346537, 0.37...",None,0.724952,...,True,300,0.1,0.0005,1,focal,2,True,5,64
0,None,0.644061,0.522721,"[0.7648818426562968, 0.4753014391287436, 0.327...",0.516068,"[0.7921313506815365, 0.4495952906548933, 0.306...",0.532099,"[0.7394447657605553, 0.5041254125412541, 0.352...",None,0.719974,...,True,300,0.1,0.0005,12,focal,2,True,5,64
0,None,0.637165,0.512580,"[0.7610513739545998, 0.4625, 0.3141891891891892]",0.506483,"[0.7869054972205065, 0.4391691394658754, 0.293...",0.521158,"[0.7368421052631579, 0.4884488448844885, 0.338...",None,0.719011,...,True,300,0.1,0.0005,123,focal,2,True,5,64


In [56]:
best_bilstm_history_80_kfold["f1"].mean()

0.519468123340839

In [57]:
best_bilstm_history_80_kfold["precision"].mean()

0.5131934561009911

In [58]:
best_bilstm_history_80_kfold["recall"].mean()

0.5285688320133344

In [59]:
np.stack(best_bilstm_history_80_kfold["f1_scores"]).mean(axis=0)

array([0.76614625, 0.46726934, 0.32498878])

In [60]:
np.stack(best_bilstm_history_80_kfold["precision_scores"]).mean(axis=0)

array([0.78972632, 0.45047958, 0.29937446])

In [61]:
np.stack(best_bilstm_history_80_kfold["recall_scores"]).mean(axis=0)

array([0.74397532, 0.4859736 , 0.35575758])

## history_length=110

In [62]:
size = 110

In [63]:
(
    bilstm_history_110_kfold,
    best_bilstm_history_110_kfold,
    _,
    __,
) = lstm_hyperparameter_search(
    history_lengths=[size],
    results_output=f"{output_dir}/lstm_history_{size}_focal_{gamma}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/lstm_history_110_focal_2_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/lstm_history_110_focal_2_kfold_best_model.csv


In [64]:
bilstm_history_110_kfold.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

/tmp/ipykernel_3407032/44427206.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  bilstm_history_110_kfold.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()


accuracy        f1  precision  \
hidden_dim dropout_rate learning_rate                                  
200        0.1          0.0001         0.639527  0.504573   0.503890   
                        0.0005         0.644828  0.523575   0.516469   
                        0.0010         0.639591  0.521153   0.513820   
           0.2          0.0001         0.640358  0.506326   0.504700   
                        0.0005         0.643167  0.524389   0.516718   
                        0.0010         0.648787  0.525656   0.520571   
300        0.1          0.0001         0.628416  0.499976   0.495022   
                        0.0005         0.637676  0.517376   0.511956   
                        0.0010         0.636718  0.519812   0.511526   
           0.2          0.0001         0.634355  0.504146   0.501033   
                        0.0005         0.635441  0.517309   0.509548   
                        0.0010         0.645466  0.523115   0.517474   
384        0.1          0.0001         0.644508  0.510684   0.508823   
                        0.0005         0.646616  0.525977   0.519497   
                        0.0010         0.645658  0.528133   0.521665   
           0.2          0.0001         0.640741  0.507691   0.504829   
                        0.0005         0.648787  0.521850   0.517352   
                        0.0010         0.637995  0.520256   0.513561   

                                         recall  valid_accuracy  valid_f1  \
hidden_dim dropout_rate learning_rate                                       
200        0.1          0.0001         0.505913        0.706969  0.614750   
                        0.0005         0.534660        0.714355  0.628064   
                        0.0010         0.533705        0.716442  0.628107   
           0.2          0.0001         0.508787        0.706166  0.614747   
                        0.0005         0.537347        0.712214  0.626349   
                        0.0010         0.534001        0.722008  0.630155   
300        0.1          0.0001         0.507195        0.701937  0.614302   
                        0.0005         0.528478        0.716228  0.628222   
                        0.0010         0.534093        0.710822  0.621957   
           0.2          0.0001         0.509318        0.704453  0.614193   
                        0.0005         0.530631        0.712963  0.626762   
                        0.0010         0.531461        0.714943  0.625680   
384        0.1          0.0001         0.513999        0.706273  0.615035   
                        0.0005         0.535767        0.714783  0.626320   
                        0.0010         0.544569        0.711732  0.621671   
           0.2          0.0001         0.511504        0.704453  0.613448   
                        0.0005         0.528465        0.721955  0.630890   
                        0.0010         0.532807        0.710822  0.620847   

                                       valid_precision  valid_recall      k  \
hidden_dim dropout_rate learning_rate                                         
200        0.1          0.0001                0.608730      0.624734  110.0   
                        0.0005                0.617569      0.644999  110.0   
                        0.0010                0.618759      0.641657  110.0   
           0.2          0.0001                0.608298      0.625635  110.0   
                        0.0005                0.615002      0.644966  110.0   
                        0.0010                0.623518      0.639415  110.0   
300        0.1          0.0001                0.604295      0.630552  110.0   
                        0.0005                0.618468      0.642221  110.0   
                        0.0010                0.612125      0.635398  110.0   
           0.2          0.0001                0.605690      0.627543  110.0   
                        0.0005                0.615929      0.645193  110.0   
                        0.0010

In [65]:
best_bilstm_history_110_kfold

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,bidirectional,hidden_dim,dropout_rate,learning_rate,seed,loss_function,gamma,k_fold,n_splits,batch_size
0,None,0.653640,0.522396,"[0.7735682819383258, 0.47458952306489444, 0.31...",0.521451,"[0.785799522673031, 0.4509658246656761, 0.3275...",0.524482,"[0.761711972238288, 0.5008250825082509, 0.3109...",None,0.729448,...,True,384,0.2,0.0005,1,focal,2,True,5,64
0,None,0.650958,0.525885,"[0.7750036933077263, 0.4638628011433238, 0.338...",0.519806,"[0.7922077922077922, 0.45917542441390463, 0.30...",0.534514,"[0.758530942741469, 0.46864686468646866, 0.376...",None,0.717405,...,True,384,0.2,0.0005,12,focal,2,True,5,64
0,None,0.641762,0.517269,"[0.7667807709480577, 0.45994420087684335, 0.32...",0.510799,"[0.7899417356639068, 0.4448727833461835, 0.297...",0.526398,"[0.7449392712550608, 0.4760726072607261, 0.358...",None,0.719011,...,True,384,0.2,0.0005,123,focal,2,True,5,64


In [66]:
best_bilstm_history_110_kfold["f1"].mean()

0.5218500556644478

In [67]:
best_bilstm_history_110_kfold["precision"].mean()

0.5173520095248619

In [68]:
best_bilstm_history_110_kfold["recall"].mean()

0.5284645873494233

In [69]:
np.stack(best_bilstm_history_110_kfold["f1_scores"]).mean(axis=0)

array([0.77178425, 0.46613218, 0.32763374])

In [70]:
np.stack(best_bilstm_history_110_kfold["precision_scores"]).mean(axis=0)

array([0.78931635, 0.45167134, 0.31106833])

In [71]:
np.stack(best_bilstm_history_110_kfold["recall_scores"]).mean(axis=0)

array([0.75506073, 0.48184818, 0.34848485])